In [28]:
# Importing libraries and packages

# %load_ext autoreload
# %autoreload 2

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.utils import shuffle
import os

print(os.getcwd())
from permutation_test_util import *

C:\Users\phuro\UCSD\Project_WHEL\data


In [29]:
# Importing dataset

os.chdir("C:/Users/phuro/UCSD/Project_WHEL/data")
path = os.path.join('interim', 'return_status_by_site.csv')
df = pd.read_csv(path, index_col=0)

In [30]:
df

,site,status
0,AZ,returned
1,CA,returned
2,CA,returned
3,OR,returned
4,CA,returned
...,...,...
2856,TX,returned
2857,TX,unreturned
2858,CA,returned
2859,CA,returned


In [47]:

def tvd_of_groups(df, groups, cats):

    '''
        groups: the binary column
        cats: the categorical column
    '''

    # Get count of each pair of catogory and group
    counts = df.pivot_table(index=cats, columns=groups, aggfunc='size')

    # Normalize each column
    distr = counts / counts.sum()

    # Compute and return the total variation distance
    return distr.diff(axis=1).iloc[:, -1].abs().sum() / 2

def permutation_simulation(df, N, shuffle_column, cats_column, 
                           significance_level):

    '''
        Run N simulations under the null hypothesis (both distributions are
        from the same population, any observed difference is due to chance)
    '''

    results = []
    for _ in range(N):
        # Shuffle group column (to_shuffle_column)
        with_shuffled = df.assign(
            shuffled=np.random.permutation(df[shuffle_column])
        )

        # Compute and store the TVD
        tvd = tvd_of_groups(with_shuffled, groups='shuffled', cats=cats_column)
        results.append(tvd)
        if _ % 10 == 0:
            print(tvd)

    # Calculate p-value and assess null hypothesis
    observed = tvd_of_groups(df, groups=shuffle_column, cats=cats_column)
    pval = (results >= observed).mean()
    if pval < significance_level:
        return f'P-value = {pval}, Reject null hypothesis at {significance_level}'
    return f'P-value = {pval}, Fail to reject null hypothesis at {significance_level}'

In [48]:
permutation_simulation(df, 10000, shuffle_column='status', cats_column='site', 
                       significance_level=0.05)

0.023394149473480286
0.03269069330861022
0.015146378476114067
0.01785911965350717
0.021267775517640593
0.0210769323019728
0.03269069330861023
0.019508673852980378
0.012910457055087421
0.011914151687492214
0.026583710407239836
0.013482410136484602
0.02125336137748443
0.01697236175109898
0.011370450320801126
0.013372862671297635
0.016209565454033903
0.021158228052453674
0.021076932301972773
0.043079252401972294
0.0491862353033427
0.03995657307853744
0.022902915576957665
0.021811476884331715
0.027851578175377423
0.034941028869793035
0.02291732971711389
0.026760139482751473
0.02445733645140017
0.021811476884331722
0.020095041074533783
0.03813058980355255
0.013020004520274367
0.04260243264560585
0.007770951240999774
0.03492661472963686
0.028409693682224728
0.01521326008643873
0.01532280755162569
0.010183301737538071
0.03600421584771287
0.018512368485385226
0.016605089459919502
0.01686281428591198
0.0491862353033427
0.021158228052453612
0.016686385210400334
0.01631911291922089
0.022902915576

'P-value = 0.0006, Reject null hypothesis at 0.05'